In [17]:
from python_graphql_client import GraphqlClient
import datetime
import pandas


# Instantiate the client with an endpoint.
clientv2 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v2/graphql")
#clientv3 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v3/graphql")

timestamp = "2021-08-30T14:45:00+02:00"
#timestamp = datetime.datetime.now().astimezone().isoformat()

# Create the query string and variables required for the request.
query = """
query ($id: String!, $timestamp: DateTime) {
  quay(id: $id) {
    estimatedCalls(numberOfDepartures: 20, startTime: $timestamp) {
      expectedArrivalTime
      aimedArrivalTime
      destinationDisplay {
        frontText
      }
      serviceJourney {
        line {
          publicCode
        }
      }
    }
  }
}


"""

# 103009 = Buran
variables = {
    "id": "NSR:Quay:103009",
    "timestamp": timestamp,
}

datav2 = clientv2.execute(query=query, variables=variables)
# datav3 = clientv3.execute(query=query, variables=variables)

#datav2

calls = datav2['data']['quay']['estimatedCalls']

output = []

for arrival in calls:
    expectedTime = arrival['expectedArrivalTime'].split('T')[1].split('+')[0]
    lineNr = arrival['serviceJourney']['line']['publicCode']
    lineName = arrival['destinationDisplay']['frontText']
    output.append([lineNr, lineName, expectedTime])
    
pandas.DataFrame(output, columns=["LinjeNr", "Navn", "Estimert tid"])

# sammenlikn med
# https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&from=Buran+1+%28Trondheim%29&to=&Time=14%3A45&Date=30.08.2021&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations=



,LinjeNr,Navn,Estimert tid
0,10,Rate via sentrum,14:45:22
1,1,Kattem via sentrum-Tiller,14:45:46
2,12,Marienborg via sentrum,14:46:14
3,1,Tillerterminalen via sentrum,14:50:00
4,2,Lund via Kolstad-Heimdal,14:50:09
5,28,Pirbadet,14:51:29
6,22,Vestlia via sentrum-Othilienborg,14:51:35
7,20,Romolslia via Tempevegen,14:52:07
8,12,Marienborg via sentrum,14:53:38
9,1,Kattem via sentrum-Tiller,14:55:45


In [13]:
calls = datav2['data']['quay']['estimatedCalls']

output = []

for arrival in calls:
    expectedTime = arrival['expectedArrivalTime'].split('T')[1].split('+')[0]
    lineNr = arrival['serviceJourney']['line']['publicCode']
    lineName = arrival['destinationDisplay']['frontText']
    output.append([lineNr, lineName, expectedTime])
    
pandas.DataFrame(output, columns=["LinjeNr", "Navn", "Estimert tid"])
    

,LinjeNr,Navn,Estimert tid
0,1,Kattem via sentrum-Tiller,14:37:05
1,20,Romolslia via Tempevegen,14:37:18
2,2,Midteggen via sentrum,14:37:49
3,1,Tillerterminalen via sentrum,14:39:46
4,2,Lund via Kolstad-Heimdal,14:40:18
5,22,Vestlia via sentrum-Othilienborg,14:41:35
6,1,Kattem via sentrum-Tiller,14:45:46
7,10,Rate via sentrum,14:45:48
8,12,Marienborg via sentrum,14:46:10
9,1,Tillerterminalen via sentrum,14:50:00
